# This notebook tries to improve how well the model describes the reality by adjusting it's weights

In [1]:
from python import *

import operator
import pickle
import os
import numpy as np
import json
from ipyparallel import Client
from itertools import repeat

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [2]:
#Create data classes
#Year, Season
seasonsData = None
#RaceId, List of tuples of (driverId, constructorId, time)
qualiResultsData = None
#DriverId, name
driversData = None
#ConstructorId, name
constructorsData = None
#EngineId, name
enginesData = None

with open('data/seasonsData.txt', 'rb') as handle:
    seasonsData = pickle.load(handle)
    #print(seasonsData)
    
with open('data/qualiResultsData.txt', 'rb') as handle:
    qualiResultsData = pickle.load(handle)
    #print(qualiResultsData)
    
with open('data/driversData.txt', 'rb') as handle:
    driversData = pickle.load(handle)
    #print(driversData)
    
with open('data/constructorsData.txt', 'rb') as handle:
    constructorsData = pickle.load(handle)
    #print(constructorsData)
    
with open('data/enginesData.txt', 'rb') as handle:
    enginesData = pickle.load(handle)
    #print(enginesData)

In [3]:
entries = []
results = []
cleaner = F1DataCleaner(seasonsData, qualiResultsData, driversData, constructorsData, enginesData)

#Constants we can change

cleaner.k_engine_change = 0.0145
cleaner.k_const_change = 0.240
cleaner.k_driver_change = 0.19
cleaner.k_const_impact = 0.80
cleaner.k_eng_impact = (1 - cleaner.k_const_impact)
cleaner.k_driver_impact = 0.02
        
cleaner.k_rookie_pwr = 0.40
#cleaner.k_rookie_variance = 5
cleaner.k_race_regress_exp = 0.87  #TODO needs to change!
#cleaner.k_variance_multiplier_end = 1.5

cleaner.k_eng_regress = 1.04
cleaner.k_const_regress = 0.60
cleaner.k_driver_regress = 0.67

cleaner.constructDataset(entries, results)
#print(entries[-10:])
#print(results[-10:])
X = np.array(entries)
y = results

forest = RandomForestRegressor(random_state=0, n_estimators=100)
forest.fit(X, y)
print(forest.feature_importances_)
#Rates based on test data!
print(forest.score(X, y))

[0.14849044 0.59296319 0.0926438  0.0467351  0.05450529 0.06466218]
0.942282940062119


In [4]:
#Linear regression
reg = LinearRegression().fit(X, y)
print("R-score: " + str(reg.score(X, y)))
print(reg.coef_)

R-score: 0.545887105757251
[0.52875555 0.83348769 0.65995298 0.11074181 0.19952604 2.1844449 ]


In [5]:
#Random forest regression

forest = RandomForestRegressor(max_depth=5, random_state=0,
                             n_estimators=100)
forest.fit(X, y)
print(forest.feature_importances_)
#Rates based on test data!
print(forest.score(X, y))

[0.10297275 0.83018667 0.03436104 0.00367642 0.00540587 0.02339726]
0.6340795432293774


# Just for testing!

In [7]:
newDrivers = json.load(open('data/newDrivers.json'))["drivers"]
newDrivers = {int(did): cid for did, cid in newDrivers.items()}

outFile = {} # The object where we write output

def getColor(constructor):
    return {
        "Mercedes": "#00d2be",
        "Ferrari": "#dc0000",
        "Red Bull": "#1e41ff",
        "Racing Point": "#f596c8",
        "Williams": "#ffffff",
        "Renault": "#fff500",
        "Toro Rosso": "#469bff",
        "Haas F1 Team": "#f0d787",
        "McLaren": "#ff8700",
        "Alfa Romeo": "#9b0000"
    }.get(constructor, "#000000")

#Manual changes
#cleaner.drivers[8].constructor = cleaner.constructors[15] #Kimi
#cleaner.drivers[841].constructor = cleaner.constructors[15] #Gio
#cleaner.drivers[844].constructor = cleaner.constructors[6] #Leclerc
#cleaner.drivers[1000] = Driver("Lando Norris", cleaner.constructors[1]) #Lando
#cleaner.drivers[832].constructor = cleaner.constructors[1] #Sainz
#cleaner.drivers[840].constructor = cleaner.constructors[10] #Stroll
cleaner.drivers[842].constructor = cleaner.constructors[5] #Gasly
#cleaner.drivers[817].constructor = cleaner.constructors[4] #Ricciardo
cleaner.drivers[848] = Driver("Alexander Albon", cleaner.constructors[9]) #Albon
#cleaner.drivers[826].constructor = cleaner.constructors[5] #Kvyat
#cleaner.drivers[1002] = Driver("George Russell", cleaner.constructors[3]) #Russell
#cleaner.drivers[9].constructor = cleaner.constructors[3] #Kubica

#cleaner.drivers[1000].pwr = cleaner.k_rookie_pwr
#cleaner.drivers[1001].pwr = cleaner.k_rookie_pwr
#cleaner.drivers[1002].pwr = cleaner.k_rookie_pwr

driversToWrite = {}
for did, cid in newDrivers.items():
    driversToWrite[int(did)] = {}
    driversToWrite[int(did)]["name"] = cleaner.drivers[int(did)].name
    driversToWrite[int(did)]["constructor"] = cleaner.drivers[int(did)].constructor.name
    driversToWrite[int(did)]["color"] = getColor(cleaner.drivers[int(did)].constructor.name)
print(driversToWrite)
outFile["drivers"] = driversToWrite

{8: {'name': 'Kimi Räikkönen', 'constructor': 'Alfa Romeo', 'color': '#9b0000'}, 841: {'name': 'Antonio Giovinazzi', 'constructor': 'Alfa Romeo', 'color': '#9b0000'}, 20: {'name': 'Sebastian Vettel', 'constructor': 'Ferrari', 'color': '#dc0000'}, 844: {'name': 'Charles Leclerc', 'constructor': 'Ferrari', 'color': '#dc0000'}, 154: {'name': 'Romain Grosjean', 'constructor': 'Haas F1 Team', 'color': '#f0d787'}, 825: {'name': 'Kevin Magnussen', 'constructor': 'Haas F1 Team', 'color': '#f0d787'}, 846: {'name': 'Lando Norris', 'constructor': 'McLaren', 'color': '#ff8700'}, 832: {'name': 'Carlos Sainz', 'constructor': 'McLaren', 'color': '#ff8700'}, 1: {'name': 'Lewis Hamilton', 'constructor': 'Mercedes', 'color': '#00d2be'}, 822: {'name': 'Valtteri Bottas', 'constructor': 'Mercedes', 'color': '#00d2be'}, 815: {'name': 'Sergio Pérez', 'constructor': 'Racing Point', 'color': '#f596c8'}, 840: {'name': 'Lance Stroll', 'constructor': 'Racing Point', 'color': '#f596c8'}, 842: {'name': 'Pierre Gasl

In [8]:
raceId = -1
with open('data/futureRaces.json', 'r') as handle:
    futureRaces = json.load(handle)
    circuit = futureRaces[0]["circuitId"]
    circuitName = futureRaces[0]["name"]
    raceId = futureRaces[0]["raceId"]
    #print(seasonsData)
    
# Edit index file
with open('../F1PredictWeb/public/data/index.json', 'r+') as handle:
    data = json.load(handle)
    data[str(futureRaces[0]["year"])][str(raceId)] = circuitName
    handle.seek(0)        # <--- should reset file position to the beginning.
    json.dump(data, handle, indent=4)
    handle.truncate()

outFile["name"] = circuitName
outFile["year"] = futureRaces[0]["year"]

In [9]:
predictedEntrants = []

for did, cid in newDrivers.items():
    print(cleaner.drivers[did].name + ": " + str(cleaner.drivers[did].pwr) + " -- " + str(cleaner.drivers[did].constructor.pwr))
    if circuit not in cleaner.drivers[did].trackpwr:
        cleaner.drivers[did].trackpwr[circuit] = 0 #TODO maybe change defaults
    if circuit not in cleaner.drivers[did].constructor.trackpwr:
        cleaner.drivers[did].constructor.trackpwr[circuit] = 0 #TODO maybe change defaults
    if circuit not in cleaner.drivers[did].constructor.engine.trackpwr:
        cleaner.drivers[did].constructor.engine.trackpwr[circuit] = 0 #TODO maybe change defaults
    
    entry = [
        cleaner.drivers[did].pwr,
        cleaner.drivers[did].constructor.pwr, 
        cleaner.drivers[did].constructor.engine.pwr,
        cleaner.drivers[did].trackpwr[circuit],
        cleaner.drivers[did].constructor.trackpwr[circuit],
        cleaner.drivers[did].constructor.engine.trackpwr[circuit]
    ]
    predictedEntrants.append(entry)

Kimi Räikkönen: -0.055712441991017145 -- 0.029102547592147997
Antonio Giovinazzi: 0.08744712500974329 -- 0.029102547592147997
Sebastian Vettel: -0.08322874252242692 -- -0.5360099619497668
Charles Leclerc: -0.16139283897752119 -- -0.5360099619497668
Romain Grosjean: 0.026143643873608104 -- -0.009761857037269558
Kevin Magnussen: 0.024404512616902817 -- -0.009761857037269558
Lando Norris: -0.012953486483177002 -- -0.028607524608592668
Carlos Sainz: -0.07983129238806108 -- -0.028607524608592668
Lewis Hamilton: -0.3058881011658354 -- -0.5163605330029643
Valtteri Bottas: -0.22767654291378794 -- -0.5163605330029643
Sergio Pérez: -0.038686055319508084 -- 0.4247120387400452
Lance Stroll: 0.19874709972615254 -- 0.4247120387400452
Pierre Gasly: 0.05175870666334699 -- 0.08824425764170743
Max Verstappen: -0.21852878763519873 -- -0.5252657756249957
Daniel Ricciardo: 0.027941389628791405 -- 0.10467039319977141
Nico Hülkenberg: 0.06271702841999487 -- 0.10467039319977141
Alexander Albon: 0 -- -0.525265

In [10]:
#Linear regression predict
predictedResults = reg.predict(np.array(predictedEntrants))
print(predictedResults)

[-0.04352021  0.05516933 -0.4395577  -0.51400998 -0.06432864 -0.0756306
 -0.02086118 -0.05002862 -0.60267314 -0.61034476  0.04389491  0.20262781
  0.28223189 -0.407628    0.11034116  0.12958874 -0.27390437  0.37880278
  0.84112904  0.96860613]


In [11]:
#Forest predict
forestResults = forest.predict(np.array(predictedEntrants))
print(forestResults)

[ 0.01388492  0.07770279 -0.61076625 -0.61922658 -0.01999479 -0.02995248
 -0.05115424 -0.07607479 -0.71364119 -0.71596007  0.16912987  0.34969218
  0.18182092 -0.54987873  0.10309128  0.15142033 -0.53181502  0.2424663
  0.96121078  1.08647502]


In [12]:
driverResults = {} # {did: {position: amount}}
orderedResults = [] # [(did, prediction) ...]
for index, (did, cid) in enumerate(newDrivers.items()):
    print("{0} ({1}): {2}".format(cleaner.drivers[int(did)].name, cleaner.drivers[int(did)].constructor.name, forestResults[index]))
    newDrivers[did] = forestResults[index]
    driverResults[int(did)] = {}
    orderedResults.append((did, forestResults[index]))
    
orderedResults.sort(key = operator.itemgetter(1))
outFile["order"] = [a for (a, b) in orderedResults]
    
for i in range(1000):
    scoreList = predictQualiResults(circuit, newDrivers)
    for i, drivRes in enumerate(scoreList):
        if i not in driverResults[drivRes[0]]:
            driverResults[drivRes[0]][i] = 0
        driverResults[drivRes[0]][i] += 1
        
for did, res in driverResults.items():
    print("{0} ({1}):".format(cleaner.drivers[int(did)].name, cleaner.drivers[int(did)].constructor.name))
    for pos, amount in sorted(res.items(), key=lambda posAmount: posAmount[0]):
        print("\t{0}: {1} %".format(pos + 1, amount / 10.0))
        
outFile["predictions"] = driverResults
print(outFile)
with open('../F1PredictWeb/public/data/' + str(raceId) + '.json', 'w') as fp:
    json.dump(outFile, fp)

Kimi Räikkönen (Alfa Romeo): 0.013884917215642927
Antonio Giovinazzi (Alfa Romeo): 0.07770279334360207
Sebastian Vettel (Ferrari): -0.610766248264182
Charles Leclerc (Ferrari): -0.6192265834341562
Romain Grosjean (Haas F1 Team): -0.019994788787949004
Kevin Magnussen (Haas F1 Team): -0.029952478545779742
Lando Norris (McLaren): -0.05115424296346575
Carlos Sainz (McLaren): -0.07607478976543725
Lewis Hamilton (Mercedes): -0.7136411949172561
Valtteri Bottas (Mercedes): -0.7159600706725839
Sergio Pérez (Racing Point): 0.16912987308137756
Lance Stroll (Racing Point): 0.349692175344089
Pierre Gasly (Toro Rosso): 0.18182092436089234
Max Verstappen (Red Bull): -0.5498787277119564
Daniel Ricciardo (Renault): 0.10309127892379463
Nico Hülkenberg (Renault): 0.1514203308080152
Alexander Albon (Red Bull): -0.5318150186593174
Daniil Kvyat (Toro Rosso): 0.24246629891099425
George Russell (Williams): 0.9612107827821036
Robert Kubica (Williams): 1.08647501588209
Kimi Räikkönen (Alfa Romeo):
	5: 0.2 %
	6: